In [ ]:
!pip install -i https://test.pypi.org/simple/ anguilla --upgrade

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import anguilla

from IPython.display import display, clear_output

import anguilla.hypervolume as hv

from anguilla.dominance import NonDominatedSet2D, non_dominated_sort
from anguilla.fitness import benchmark

print(anguilla.__version__)

0.0.14


In [4]:
PAIRS = [
    ('ZDT1', 2),
    ('ZDT2', 2),
    ('ZDT3', 2),
    ('ZDT4', 2),
    ('ZDT6', 2),
    ('IHR1', 2),
    ('IHR2', 2),
    ('IHR3', 2),
    ('IHR4', 2),
    ('IHR6', 2),
    ('ELLI1', 2),
    ('ELLI2', 2),
    ('CIGTAB1', 2),
    ('CIGTAB2', 2),
    ('DTLZ1', 3),
    ('DTLZ2', 3),
    ('DTLZ3', 3),
    ('DTLZ4', 3),
    ('DTLZ5', 3),
    ('DTLZ6', 3),
    ('DTLZ7', 3),
    ('GELLI', 3)
]

ALGOS = [
    '(100+1)-MO-CMA-ES-I',
    '(100+1)-MO-CMA-ES-P',
    '(100+100)-MO-CMA-ES-I',
    '(100+100)-MO-CMA-ES-P'
]

In [5]:
class NonDominatedSetKD:
    def __init__(self):
        self.union = []
    
    def insert(self, points):
        self.union.append(points)
        
    @property
    def upper_bound(self):
        points = np.vstack(self.union)
        ranks, _ = non_dominated_sort(points, 1)
        nondominated = points[ranks == 1]
        reference = np.max(nondominated, axis=0)
        return reference

In [7]:
def check_exists(basepath, func):
    basepath = pathlib.Path(basepath)
    paths = list(basepath.glob('**/{}*.fitness.csv'.format(func)))
    return len(paths) != 0

def get_fn_glob(basepath, func, n_evaluations):
    basepath = pathlib.Path(basepath)
    paths = basepath.glob('**/{}*_{}.fitness.csv'.format(func, n_evaluations))
    return paths

def get_fn_algo_glob(basepath, func, algo, n_evaluations):
    basepath = pathlib.Path(basepath)
    paths = basepath.glob('**/{}_{}_*_{}.fitness.csv'.format(func, algo, n_evaluations))
    return paths

def get_fn_reference_point(basepath, func, n_objs, n_evaluations):
    paths = get_fn_glob(basepath, func, n_evaluations)
    point_set = NonDominatedSet2D() if n_objs == 2 else NonDominatedSetKD()
    for path in paths:
        points = np.genfromtxt(path, delimiter=',')
        point_set.insert(points)
    reference = point_set.upper_bound + 1.0
    return reference

def compute_fn_hv_indicators(reference, basepath, func, algo, n_objs, n_evaluations):
    paths = get_fn_algo_glob(basepath, func, algo, n_evaluations)
    indicators = []
    for path in paths:
        points = np.genfromtxt(path, delimiter=',')
        display('Computing HV')
        indicator = hv.calculate(points, reference)
        display('Done')
        indicators.append(indicator)
    return np.array(indicators)

def compute_summaries(basepath, n_evaluations):
    refs = {}
    rows = []
    for func, n_objs in PAIRS:
        if check_exists(basepath, func):
            display('Computing reference point for: "{}, {}, {}"'.format(func, n_objs, n_evaluations))
            reference = get_fn_reference_point(basepath, func, n_objs, n_evaluations)
            row = []
            for algo in ALGOS:
                indicators = compute_fn_hv_indicators(reference, basepath, func, algo, n_objs, n_evaluations)
                median_indicator = np.median(indicators)
                display('  Computing HV indicator for "{}, {}, {}, {}": {}'.format(func, n_objs, algo, n_evaluations, median_indicator))
                row.append(median_indicator)
            rows.append(row)
            clear_output()
    df = pd.DataFrame(rows)
    df.columns = ALGOS
    return df

In [ ]:
compute_summaries('data/shark', 50000)

In [ ]:
compute_summaries('data/anguilla', 50000)

'Computing reference point for: "DTLZ1, 3, 50000"'

'Computing HV'

'Done'